In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
numeric_cols = df.drop(columns=['converted']).select_dtypes(include=['int64', 'float64']).columns 
categorical_cols = df.select_dtypes(include=['object']).columns

In [5]:
df.isna().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [6]:
for n in numeric_cols:
    df[n] = df[n].fillna(0)

for c in categorical_cols:
    df[c] = df[c].fillna('NA')

# Q-1

In [7]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

# Q-2

In [8]:
correlation_matrix = df[numeric_cols].corr().abs()
correlation_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,0.023565,0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,0.023565,0.027036,1.000000,0.009888
lead_score,0.004879,0.015610,0.009888,1.000000


In [9]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

x_train = df_train.drop(columns=['converted'])
x_val = df_val.drop(columns=['converted'])
x_test = df_test.drop(columns=['converted'])

# Q-3

In [10]:
from sklearn.metrics import mutual_info_score

for c in categorical_cols:
    print( c , round(mutual_info_score(df_train[c], y_train),2))


lead_source 0.04
industry 0.01
employment_status 0.01
location 0.0


# Q-4

In [11]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[list(categorical_cols)].to_dict(orient='records')
x_train = dv.fit_transform(train_dict)

val_dict = df_val[list(categorical_cols)].to_dict(orient='records')
x_val = dv.transform(val_dict)


In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model.fit(x_train, y_train)
y_pred = model.predict_proba(x_val)[:, 1]
decision = y_pred >= 0.5
accuracy = (decision == y_val).mean()
print('Accuracy:', accuracy.round(2))

Accuracy: 0.61


# Q-5

In [26]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model.fit(x_train, y_train)
y_pred = model.predict_proba(x_val)[:, 1]
decision = y_pred >= 0.5
accuracy = (decision == y_val).mean()
print('Accuracy:', accuracy)
main_accuracy = accuracy

Accuracy: 0.6075085324232082


In [22]:
df_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03,0
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77,1
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59,1
3,NA,technology,1,74956.0,employed,europe,3,0.34,1
4,organic_search,retail,3,59335.0,student,australia,1,0.98,1
...,...,...,...,...,...,...,...,...,...
871,organic_search,other,1,43907.0,employed,australia,4,0.33,1
872,social_media,retail,3,64969.0,employed,north_america,1,0.18,0
873,NA,education,3,89042.0,employed,asia,4,0.75,1
874,social_media,manufacturing,1,0.0,self_employed,europe,1,0.65,0


In [32]:
exluded_cols = ["industry","employment_status","lead_score"]

for col in exluded_cols:
    df_train_excluded = df_train.drop(columns=[col])
    df_val_excluded = df_val.drop(columns=[col])

    dv_excluded = DictVectorizer(sparse=False)
    
    new_categorical_cols = [c for c in categorical_cols if c != col]
    train_dict = df_train_excluded[list(new_categorical_cols)].to_dict(orient='records')
    x_train = dv_excluded.fit_transform(train_dict)

    val_dict = df_val_excluded[list(new_categorical_cols)].to_dict(orient='records')
    x_val = dv_excluded.transform(val_dict)

    model_excluded = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_excluded.fit(x_train, y_train)

    y_pred = model_excluded.predict_proba(x_val)[:, 1]
    decision = y_pred >= 0.5
    accuracy = (decision == y_val).mean()
    print(f'Excluding {col}, Accuracy: {accuracy.round(3)}')
    print("The difference with the main accuracy", abs(main_accuracy - accuracy).round(3))
    print("\n")
    

Excluding industry, Accuracy: 0.604
The difference with the main accuracy 0.003


Excluding employment_status, Accuracy: 0.546
The difference with the main accuracy 0.061


Excluding lead_score, Accuracy: 0.608
The difference with the main accuracy 0.0




# Q-6

In [39]:
C = [0.01, 0.1, 1, 10, 100]
for c in C:
    model_c = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model_c.fit(x_train, y_train)

    y_pred = model_c.predict_proba(x_val)[:, 1]
    decision = y_pred >= 0.5
    accuracy = (decision == y_val).mean()
    print(f'For C={c}, Accuracy: {accuracy.round(3)}')

For C=0.01, Accuracy: 0.56
For C=0.1, Accuracy: 0.601
For C=1, Accuracy: 0.608
For C=10, Accuracy: 0.604
For C=100, Accuracy: 0.604
